In [123]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os

### Compilation of all appropriate CSV files

**More info to be added soon

In [124]:
#cleanup
path_to_parent = "/CSV"
current_path = os.getcwd()

# Initializing placeholder dataframe
copy_df = pd.DataFrame()

# Initializing dicts
balance_sheet_df_collection = {}
cashflow_statement_df_collection = {}
income_statement_df_collection = {}
market_stats_df_collection = {}

# Initializing dfs
balance_sheet_df = pd.DataFrame()
cashflow_statement_df = pd.DataFrame()
income_statement_df = pd.DataFrame()
market_stats_df = pd.DataFrame()

#Iterate through balance sheets quarterly
for folder in os.listdir(current_path+path_to_parent):
    for csv_file in os.listdir(os.path.join(current_path+path_to_parent, folder)):

        path_name = current_path+path_to_parent+'/'+folder+'/'+csv_file
       
        if csv_file == "balance_sheet_quarterly.csv":
            # Setting up a transient dataframe that gets rewritten with every iteration
            copy_df = pd.read_csv(path_name)
            # Setting up index for financial institutions
            copy_df["Unnamed: 0"] = (copy_df["Unnamed: 0"] >= 0).astype(int).replace(1,folder)
            # Setting up all columns to create a dictionary of corresponding dataframes
            column_names = copy_df.columns
            balance_sheet_df_collection[folder] = pd.DataFrame(copy_df, columns= column_names)
        elif csv_file == "cashflow_statement_quarterly.csv":
            # Setting up a transient dataframe that gets rewritten with every iteration
            copy_df = pd.read_csv(path_name)
            # Setting up index for financial institutions
            copy_df["Unnamed: 0"] = (copy_df["Unnamed: 0"] >= 0).astype(int).replace(1,folder)
            # Setting up all columns to create a dictionary of corresponding dataframes
            column_names = copy_df.columns
            cashflow_statement_df_collection[folder] = pd.DataFrame(copy_df, columns= column_names)
        elif csv_file == "income_statement_quarterly.csv":
            # Setting up a transient dataframe that gets rewritten with every iteration
            copy_df = pd.read_csv(path_name)
            # Setting up index for financial institutions
            copy_df["Unnamed: 0"] = (copy_df["Unnamed: 0"] >= 0).astype(int).replace(1,folder)
            # Setting up all columns to create a dictionary of corresponding dataframes
            column_names = copy_df.columns
            income_statement_df_collection[folder] = pd.DataFrame(copy_df, columns= column_names)
        elif csv_file == "market_stats.csv":
            # Setting up a transient dataframe that gets rewritten with every iteration
            copy_df = pd.read_csv(path_name)
            # Setting up index for financial institutions
            copy_df["Unnamed: 0"] = (copy_df["Unnamed: 0"] >= 0).astype(int).replace(1,folder)
            # Setting up all columns to create a dictionary of corresponding dataframes
            column_names = copy_df.columns
            market_stats_df_collection[folder] = pd.DataFrame(copy_df, columns= column_names)
        else:
            pass


balance_sheet_df = pd.concat(balance_sheet_df_collection)
cashflow_statement_df = pd.concat(cashflow_statement_df_collection)
income_statement_df = pd.concat(income_statement_df_collection)
market_stats_df = pd.concat(market_stats_df_collection)

#balance_sheet_df.head(200)
#cashflow_statement_df.head(200)
#income_statement_df.head(200)
#market_stats_df.head(200)



#def create_balance_sheet_df(path_name, folder, csv_file):
#    if csv_file == "balance_sheet_quarterly.csv":
#        # Setting up a transient dataframe that gets rewritten with every iteration
#        copy_df = pd.read_csv(path_name)
#        # Setting up index for financial institutions
#        copy_df["Unnamed: 0"] = (copy_df["Unnamed: 0"] >= 0).astype(int).replace(1,folder)
#        # Setting up all columns to create a dictionary of corresponding dataframes
#        column_names = copy_df.columns
#        balance_sheet_df_collection[folder] = pd.DataFrame(copy_df, columns= column_names)
#    return balance_sheet_df_collection




### Clean up of compiled CSV files

 - balance_sheet_df
 - cashflow_statement_df
 - income_statement_df
 - market_stats_df

### Balance Sheet Data Cleanup

In [ ]:

balance_sheet_df.rename(columns = {"Unnamed: 0":"Ticker", "Breakdown":"Date"}, inplace = True)
balance_sheet_df["Date"] = pd.to_datetime(balance_sheet_df["Date"])
balance_sheet_df.index.set_names(['Ticker Symbol', 'Index'], inplace = True) # Set index names 

# Clean column types
cols=[i for i in balance_sheet_df.columns if i not in ["Ticker","Date"]]
for col in cols:
    balance_sheet_df[col] = balance_sheet_df[col].astype(str).str.replace(',','')
    balance_sheet_df[col].replace('-', np.nan, inplace = True) #double check no negative values, find way to replace "-" for null values NaN without replacing negative values
    balance_sheet_df[col]= balance_sheet_df[col].astype(float)

balance_sheet_df

### Cashflow Statement Data Cleanup

In [ ]:
cashflow_statement_df.rename(columns = {"Unnamed: 0":"Ticker", "Breakdown":"Date"}, inplace = True)
cashflow_statement_df.index.set_names(['Ticker Symbol', 'Index'], inplace = True) # Set index names 
cashflow_statement_df.drop(0, level=1, axis=0, inplace=True) # Erase all ttm values as not necessary (trailing twelve months data not necessary)
cashflow_statement_df["Date"] = pd.to_datetime(cashflow_statement_df["Date"]) # Updating column type to datetime

# Clean column types
cols=[i for i in cashflow_statement_df.columns if i not in ["Ticker","Date"]]
for col in cols:
    cashflow_statement_df[col] = cashflow_statement_df[col].astype(str).str.replace(',','')
    cashflow_statement_df[col].replace('-', np.nan, inplace = True) #double check no negative values, find way to replace "-" for null values NaN without replacing negative values but it works?
    cashflow_statement_df[col]= cashflow_statement_df[col].astype(float)

cashflow_statement_df

### Income Statement Data Cleanup

In [ ]:
income_statement_df.rename(columns = {"Unnamed: 0":"Ticker", "Breakdown":"Date"}, inplace = True)
income_statement_df.index.set_names(['Ticker Symbol', 'Index'], inplace = True) # Set index names 
income_statement_df.drop(0, level=1, axis=0, inplace=True) # Erase all ttm values as not necessary (trailing twelve months data not necessary)
income_statement_df["Date"] = pd.to_datetime(income_statement_df["Date"]) # Updating column type to datetime

# Clean column types
cols=[i for i in income_statement_df.columns if i not in ["Ticker","Date"]]
for col in cols:
    income_statement_df[col] = income_statement_df[col].astype(str).str.replace(',','')
    income_statement_df[col].replace('-', np.nan, inplace = True) #double check no negative values, find way to replace "-" for null values NaN without replacing negative values but it works?
    income_statement_df[col]= income_statement_df[col].astype(float)

income_statement_df

### Market Stats Data Cleanup

In [122]:
market_stats_df.rename(columns = {"Unnamed: 0":"Ticker", "Breakdown":"Date", "Market Cap (intraday)":"Market Cap (intraday) [Billions]"}, inplace = True)
market_stats_df.index.set_names(['Ticker Symbol', 'Index'], inplace = True) # Set index names 
market_stats_df.drop(0, level=1, axis=0, inplace=True) # Erase all "as of date" rows (1st row for every ticker - unnecessary data)
market_stats_df["Date"] = pd.to_datetime(market_stats_df["Date"]) # Updating column type to datetime

# Clean column types (all numerical columns except market cap are floats)
#cols=[i for i in market_stats_df.columns if i not in ["Ticker","Date"]]
#for col in cols:
#    market_stats_df[col].replace('-', np.nan, inplace = True) #double check no negative values, find way to replace "-" for null values NaN without replacing negative values but it works?

# replace "." with "" in only market cap and replace M with 4 zeros, replace B with 7 zeros
market_stats_df["Market Cap (intraday) [Billions]"].replace(".","")
#market_stats_df["Market Cap (intraday) [Billions]"].str.replace("B", "0000000", inplace=True)
#market_stats_df["Market Cap (intraday) [Billions]"].str.replace("M", "0000", inplace=True)

#if there is a B replace it with "" and then turn it into an integer then multiply it by 1,000,000,000
# or keep it as billions and convert only M into decimal billions
market_stats_df

Ticker       Date Market Cap (intraday) [Billions]   
Ticker Symbol Index                                                      
EWBC          1       EWBC 2023-03-31                            7.82B  \
              2       EWBC 2022-12-31                            9.29B   
              3       EWBC 2022-09-30                            9.46B   
              4       EWBC 2022-06-30                            9.13B   
              5       EWBC 2022-03-31                           11.24B   
...                    ...        ...                              ...   
SIVBQ         137    SIVBQ 1988-12-31                           28.93M   
              138    SIVBQ 1988-09-30                           14.21M   
              139    SIVBQ 1988-06-30                           13.24M   
              140    SIVBQ 1988-03-31                           10.25M   
              141    SIVBQ 1987-12-31                           10.15M   

                     Enterprise Value  Trailing P/E  Forward P/E   
Ticker Symbol Index                                                
EWBC          1                   NaN          7.01         6.19  \
              2                   NaN          9.32         7.71   
              3                   NaN         10.23         7.64   
              4                   NaN         10.25         8.65   
              5                   NaN         12.95        11.21   
...                               ...           ...          ...   
SIVBQ         137                 NaN          7.84          NaN   
              138                 NaN          6.13          NaN   
              139                 NaN          6.47          NaN   
              140                 NaN          5.78          NaN   
              141                 NaN          6.77          NaN   

                     PEG Ratio (5 yr expected)  Price/Sales  Price/Book   
Ticker Symbol Index                                                       
EWBC          1                            NaN         3.41        1.31  \
              2                           0.24         4.42        1.64   
              3                           0.24         4.87        1.69   
              4                            NaN         4.97        1.60   
              5                            NaN         6.28        1.93   
...                                        ...          ...         ...   
SIVBQ         137                          NaN          NaN         NaN   
              138                          NaN          NaN         NaN   
              139                          NaN          NaN         NaN   
              140                          NaN          NaN         NaN   
              141                          NaN          NaN         NaN   

                     Enterprise Value/Revenue  Enterprise Value/EBITDA  
Ticker Symbol Index                                                     
EWBC          1                           NaN                      NaN  
              2                           NaN                      NaN  
              3                           NaN                      NaN  
              4                           NaN                      NaN  
              5                           NaN                      NaN  
...                                       ...                      ...  
SIVBQ         137                         NaN                      NaN  
              138                         NaN                      NaN  
              139                         NaN                      NaN  
              140                         NaN                      NaN  
              141                         NaN                      NaN  

[3334 rows x 11 columns]

In [111]:
market_stats_df

Unnamed: 0                          Date Market Cap (intraday)   
EWBC  0         EWBC  As of Date: 4/19/2023Current                 7.87B  \
      1         EWBC                     3/31/2023                 7.82B   
      2         EWBC                    12/31/2022                 9.29B   
      3         EWBC                     9/30/2022                 9.46B   
      4         EWBC                     6/30/2022                 9.13B   
...              ...                           ...                   ...   
SIVBQ 137      SIVBQ                    12/31/1988                28.93M   
      138      SIVBQ                     9/30/1988                14.21M   
      139      SIVBQ                     6/30/1988                13.24M   
      140      SIVBQ                     3/31/1988                10.25M   
      141      SIVBQ                    12/31/1987                10.15M   

           Enterprise Value  Trailing P/E  Forward P/E   
EWBC  0                 NaN          6.52         6.37  \
      1                 NaN          7.01         6.19   
      2                 NaN          9.32         7.71   
      3                 NaN         10.23         7.64   
      4                 NaN         10.25         8.65   
...                     ...           ...          ...   
SIVBQ 137               NaN          7.84          NaN   
      138               NaN          6.13          NaN   
      139               NaN          6.47          NaN   
      140               NaN          5.78          NaN   
      141               NaN          6.77          NaN   

           PEG Ratio (5 yr expected)  Price/Sales  Price/Book   
EWBC  0                          NaN         3.19        1.25  \
      1                          NaN         3.41        1.31   
      2                         0.24         4.42        1.64   
      3                         0.24         4.87        1.69   
      4                          NaN         4.97        1.60   
...                              ...          ...         ...   
SIVBQ 137                        NaN          NaN         NaN   
      138                        NaN          NaN         NaN   
      139                        NaN          NaN         NaN   
      140                        NaN          NaN         NaN   
      141                        NaN          NaN         NaN   

           Enterprise Value/Revenue  Enterprise Value/EBITDA  
EWBC  0                         NaN                      NaN  
      1                         NaN                      NaN  
      2                         NaN                      NaN  
      3                         NaN                      NaN  
      4                         NaN                      NaN  
...                             ...                      ...  
SIVBQ 137                       NaN                      NaN  
      138                       NaN                      NaN  
      139                       NaN                      NaN  
      140                       NaN                      NaN  
      141                       NaN                      NaN  

[3364 rows x 11 columns]

In [ ]:
market_stats_df["Unnamed: 0"].unique()

In [112]:
market_stats_df.dtypes

Unnamed: 0                    object
Date                          object
Market Cap (intraday)         object
Enterprise Value             float64
Trailing P/E                 float64
Forward P/E                  float64
PEG Ratio (5 yr expected)    float64
Price/Sales                  float64
Price/Book                   float64
Enterprise Value/Revenue     float64
Enterprise Value/EBITDA      float64
dtype: object

### Loading objects/csv files to AWS S3

In [132]:
import boto3

access_key = os.environ.get('AWS_ACCESS_KEY_ID')
secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

def get_file_paths(folder_path):
    file_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.csv'):  # Filter CSV files
                file_path = os.path.join(root, file)
                file_paths.append(file_path)
    return file_paths

def upload_files_to_s3(file_paths, bucket_name):
    s3 = boto3.client('s3', aws_access_key_id=access_key, aws_secret_access_key=secret_key)
    for file_path in file_paths:
        file_name = file_path.split('/')[-1]  # Extract the file name from the path
        s3.upload_file(file_path, bucket_name, file_name)
        print(f"Uploaded {file_name} to {bucket_name}")

# Folder path containing the CSV files
folder_path = os.getcwd() + "/compiled_CSV"

# Call the function to get file paths
file_paths = get_file_paths(folder_path)

# Name of the S3 bucket
bucket_name = 'ds4a-c1-team21'

# Call the function to upload files
upload_files_to_s3(file_paths, bucket_name)

Uploaded balance_sheet.csv to ds4a-c1-team21
Uploaded market_stats.csv to ds4a-c1-team21
Uploaded income_statement.csv to ds4a-c1-team21
Uploaded cashflow_statement.csv to ds4a-c1-team21
